# Taxonomy classification

# 1. Download and construct reference database 


## 1.1 Download reference database

In [1]:
! qiime rescript get-silva-data \
    --p-version '138' \
    --p-target 'SSURef_NR99' \
    --p-include-species-labels \
    --o-silva-sequences Data/3-silva-138-ssu-nr99-seqs.qza \
    --o-silva-taxonomy Data/3-silva-138-ssu-nr99-tax.qza

Saved FeatureData[RNASequence] to: Data/3-silva-138-ssu-nr99-seqs.qza
Saved FeatureData[Taxonomy] to: Data/3-silva-138-ssu-nr99-tax.qza


## 1.2 Database curation

In [1]:
! qiime rescript cull-seqs \
    --i-sequences Data/3-silva-138-ssu-nr99-seqs.qza \
    --p-num-degenerates 5 \
    --p-homopolymer-length 8 \
    --p-n-jobs 3 \
    --o-clean-sequences Data/3-silva-138-ssu-nr99-seqs-cleaned.qza

Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-cleaned.qza


In [1]:
! qiime rescript filter-seqs-length-by-taxon \
    --i-sequences Data/3-silva-138-ssu-nr99-seqs-cleaned.qza \
    --i-taxonomy Data/3-silva-138-ssu-nr99-tax.qza \
    --p-labels Archaea Bacteria Eukaryota \
    --p-min-lens 900 1200 1400 \
    --o-filtered-seqs Data/3-silva-138-ssu-nr99-seqs-filt.qza \
    --o-discarded-seqs Data/3-silva-138-ssu-nr99-seqs-discard.qza

Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-filt.qza
Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-discard.qza


In [1]:
! qiime rescript dereplicate \
    --i-sequences Data/3-silva-138-ssu-nr99-seqs-filt.qza  \
    --i-taxa Data/3-silva-138-ssu-nr99-tax.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences Data/3-silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --o-dereplicated-taxa Data/3-silva-138-ssu-nr99-tax-derep-uniq.qza

Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-derep-uniq.qza
Saved FeatureData[Taxonomy] to: Data/3-silva-138-ssu-nr99-tax-derep-uniq.qza


## 1.3 PCR-region extraction

### 1.3.1 Bacteria (27f-338r)

In [4]:
! qiime feature-classifier extract-reads \
    --i-sequences Data/3-silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --p-f-primer AGRGTTHGATYMTGGCTCAG \
    --p-r-primer GCTGCCTCCCGTAGGAGT \
    --p-n-jobs 3 \
    --p-read-orientation 'forward' \
    --o-reads Data/3-silva-138-ssu-nr99-seqs-27f-338r.qza

Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-27f-338r.qza


In [1]:
! qiime rescript dereplicate \
    --i-sequences Data/3-silva-138-ssu-nr99-seqs-27f-338r.qza \
    --i-taxa Data/3-silva-138-ssu-nr99-tax-derep-uniq.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences Data/3-silva-138-ssu-nr99-seqs-27f-338r-uniq.qza \
    --o-dereplicated-taxa  Data/3-silva-138-ssu-nr99-tax-27f-338r-derep-uniq.qza

Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-27f-338r-uniq.qza
Saved FeatureData[Taxonomy] to: Data/3-silva-138-ssu-nr99-tax-27f-338r-derep-uniq.qza


### 1.3.2 Archaea (arch349F-arch806R)

In [2]:
! qiime feature-classifier extract-reads \
    --i-sequences Data/3-silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --p-f-primer GYGCASCAGKCGMGAAW \
    --p-r-primer GGACTACVSGGGTATCTAAT \
    --p-n-jobs 3 \
    --p-read-orientation 'forward' \
    --o-reads Data/3-silva-138-ssu-nr99-seqs-349f-806r.qza

Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-349f-806r.qza


In [3]:
! qiime rescript dereplicate \
    --i-sequences Data/3-silva-138-ssu-nr99-seqs-349f-806r.qza \
    --i-taxa Data/3-silva-138-ssu-nr99-tax-derep-uniq.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences Data/3-silva-138-ssu-nr99-seqs-349f-806r-uniq.qza \
    --o-dereplicated-taxa  Data/3-silva-138-ssu-nr99-tax-349f-806r-derep-uniq.qza

Saved FeatureData[Sequence] to: Data/3-silva-138-ssu-nr99-seqs-349f-806r-uniq.qza
Saved FeatureData[Taxonomy] to: Data/3-silva-138-ssu-nr99-tax-349f-806r-derep-uniq.qza


# 2. Taxonomy assignment of bacteria 

## 2.1 Taxonomy classifier for region 27f-308r

In [4]:
! qiime feature-classifier fit-classifier-naive-bayes \
    --i-reference-reads Data/3-silva-138-ssu-nr99-seqs-27f-338r-uniq.qza \
    --i-reference-taxonomy Data/3-silva-138-ssu-nr99-tax-27f-338r-derep-uniq.qza \
    --o-classifier Data/3-classifier-27f-308r.qza

## 2.1 Taxonomy Assignment

In [6]:
! qiime feature-classifier classify-sklearn \
    --i-classifier Data/3-classifier-27f-308r.qza \
    --i-reads Data/1-rep-seqs_bac.qza \
    --o-classification Results/3-taxonomy_bac.qza \
    --p-n-jobs 3

# 3. Taxonomy assignment of archaea

## 3.1 Taxonomy classifier for region archaea349f-archaea806r

In [5]:
! qiime feature-classifier fit-classifier-naive-bayes \
    --i-reference-reads Data/3-silva-138-ssu-nr99-seqs-349f-806r-uniq.qza \
    --i-reference-taxonomy Data/3-silva-138-ssu-nr99-tax-349f-806r-derep-uniq.qza \
    --o-classifier Data/3-classifier-349f-806r.qza

Usage: qiime feature-classifier fit-classifier-naive-bayes 
           [OPTIONS]

  Create a scikit-learn naive_bayes classifier for reads

Inputs:
  --i-reference-reads ARTIFACT FeatureData[Sequence]
                                                                    [required]
  --i-reference-taxonomy ARTIFACT FeatureData[Taxonomy]
                                                                    [required]
  --i-class-weight ARTIFACT FeatureTable[RelativeFrequency]
                                                                    [optional]
Parameters:
  --p-classify--alpha NUMBER
                                                              [default: 0.001]
  --p-classify--chunk-size INTEGER
                                                              [default: 20000]
  --p-classify--class-prior TEXT
                                                             [default: 'null']
  --p-classify--fit-prior / --p-no-classify--fit-prior
                                             

## 2.1 Taxonomy Assignment

In [4]:
!qiime feature-classifier classify-sklearn

Usage: qiime feature-classifier classify-sklearn [OPTIONS]

  Classify reads by taxon using a fitted classifier.

Inputs:
  --i-reads ARTIFACT FeatureData[Sequence]
                         The feature data to be classified.         [required]
  --i-classifier ARTIFACT
    TaxonomicClassifier  The taxonomic classifier for classifying the reads.
                                                                    [required]
Parameters:
  --p-reads-per-batch VALUE Int % Range(1, None) | Str % Choices('auto')
                         Number of reads to process in each batch. If "auto",
                         this parameter is autoscaled to min( number of query
                         sequences / n-jobs, 20000).         [default: 'auto']
  --p-n-jobs INTEGER     The maximum number of concurrently worker processes.
                         If -1 all CPUs are used. If 1 is given, no parallel
                         computing code is used at all, which is useful for
                       